# Øvelser til kodeklubben 2.0
I dette øvelsessæt skal vi scrape og rense et datasæt med danske politikeres holdninger til en hel række holdninger. Datasættet kommer fra DR, som i forbindelse med kommunalvalget i 2017 har bedt alle kandidater om at udfylde en kandidat-test. Vi udnytter at testen sammenligner vores svar med svarende fra samtlige kandidater på en måde hvor vi kan tilgå kandidaternes svar på hvert spørgsmål. 

Som udgangspunkt skal i selv skrive alt koden - hvis der i bestemte øvelser skal skrives meget kode giver vi jer nogle *helper-functions*. 

## Opgave 1 - *scrape* data fra DR

Opgaven har en del trin vi skal igennem. 
* Først finder vi en liste over alle landets kommuner.
* Så henter vi data for alle kommunalpolitikere i hver enkelt kommune.
* Til sidst renser vi vores datasæt

### 1.1 Hvordan *server* DR.dk deres data? 
I skal ikke skrive noget kode i den første opgave. Gå ind på [kandidattesten](https://www.dr.dk/nyheder/politik/kv17/kandidat-testen). Åben din browsers *developer console* (CTRL+SHIFT+I), find fanen *network*. Besvar spørgsmålene i kandidattesten og tryk på **Se resultat**. 

Find nu de .json filer DR's server har sendt til jeres computer (*hint:* kig på developer consolen). 

I skal lede efter en json fil der indeholder en oversigt over alle kommunerne. Noter linket til den json, i næste opgave skal vi downloade den, sådan at vi kan fremsøge kandidater for hver enkelt kommune automatisk. 

```
```

### 1.2 Hent json-filen med kommune-ID'er 
I skal bruge det link i fandt i 2.1.1. Brug `request` modulet til at downloade json filen. 

In [1]:
import requests 

url = 'https://www.dr.dk/tjenester/kv17-candidateapi/api/constituency/all'

response = requests.get(url)
kommuner = response.json()

### 1.3 Udtræk de relevante oplysninger fra json filen
Json filen i har hentet i spørgsmål 2.1.2 indeholder en del unødvendig information for vores formål. Kig på DR-hjemmesiden og find ud af hvilket felt i filen vi skal bruge når vi skal loope igennem alle kandidater i alle kommuner. Skriv noget kode der genererer en liste med de nødvendige data. (*Hint: det i skal bruge er en liste over de slugs DR bruger som id for hver at landets kommuner)*.  
> *Note:* en *slug* er bare et element af en url, foreksempel er /farvel/ slug i www.hej.dk/farvel/.

In [2]:
kommuneslugs = [x['slug'] for x in kommuner['constituencies']]

### 1.4 Skriv en funktion der downloader alle kandidater fra en enkelt kommune.
Nu har i en liste over samtlige kommuner i landet, og deres *slugs*. I stedet for at downloade data for alle kommunerne på én gang, skal i skrive en funktion er downloader for en enkelt (variabel) kommune. Funktionen skal tage ét argument `kommuneslug`, så kan i nemlig nemt køre funktionen på alle kommunerne bagefter. 

Klik tilbage til din besvarelse af kandidattesten og klik på *"se alle kandidater i x kommune"*. Ligesom i opgave 2.1.1 sender https://www.dr.dk/nyheder/politik/kv17/se-din-stemmeseddel?kommune=kalundborg en json fil. Denne fil indeholder en lang række data på kandidaterne i kalundborg kommune, jeres funktion skal downloade denne json fil.
> *Hint:* i skal have gang i jeres developer console igen

Når i har skrevet funktionen bør i teste den, for eksempel på *kalundborg* kommune. 



In [13]:
def hent_kommune_data(kommuneslug):
    url = f'https://www.dr.dk/tjenester/kv17-candidateapi/api/ballot/{kommuneslug}'    
    response = requests.get(url)
 
    if response.ok:
        return response.json()
    else:
        return None

In [4]:
kalund = hent_kommune_data('kalundborg')
kalund

{'ballot': {'constituency': {'id': 33,
   'name': 'Kalundborg',
   'slug': 'kalundborg',
   'greaterConstituencyId': 1},
  'ballotParties': [{'partyId': 1,
    'partyName': 'Socialdemokratiet',
    'partyLetter': 'A',
    'candidates': [{'id': 11473,
      'firstName': 'Gunver',
      'lastName': 'Jensen',
      'photoUrl': 'http://www.altinget.dk/images/kv17/kvadrat-11473-gunver-jensen-a-721.jpg',
      'partyLetter': 'A',
      'partyName': 'Socialdemokratiet',
      'isPartyLeader': False,
      'answerString': '111255531144343'},
     {'id': 16874,
      'firstName': 'Lars Kuhre',
      'lastName': 'Mortensen',
      'photoUrl': 'http://www.altinget.dk/images/kv17/kvadrat-16874-Lars-Kuhre-Mortensen-a.jpg',
      'partyLetter': 'A',
      'partyName': 'Socialdemokratiet',
      'isPartyLeader': False,
      'answerString': '141455543235335'},
     {'id': 16867,
      'firstName': 'Kirsten',
      'lastName': 'Rask',
      'photoUrl': 'http://www.altinget.dk/images/kv17/kvadrat-16867

### 1.5 Rens datasættet for en enkelt kommune
Før vi downloader data for hele landet skal vi have en process sat op som automatisk renser rådata og placerer dem i en [pandas DataFrame](http://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.html). Skriv derfor en funktion der tager den rå json fil indeholdende én kommunes kandidater som indput, og konverterer denne til en DataFrame.
> *Hint:* Der skal skrives lidt mere kode her end tidligere (se evt slides)

In [16]:
import pandas as pd 

def rens_kommune_data(kommunejson):
    if kommunejson is None:
        print('Not a valid json')
        return None
    
    ballot = kommunejson['ballot']['constituency']
    parties = kommunejson['ballot']['ballotParties']
    
    _name = ballot['slug']
    _id = ballot['id']
    
    for party in parties:                
        party_candidates = party['candidates']
        
        for candidate in party_candidates:
            
            _letter = candidate['partyLetter']
            _pname = candidate['partyName']       
            _fname = candidate['firstName']
            _lname = candidate['lastName']
            _ans = candidate['answerString']
            
            data = {'id': [_id],
                    'kommune': [_name],
                    'letter': [_letter],
                    'party': [_pname],
                    'name': ['{} {}'.format(_fname, _lname)],
                    'answers': [_ans]
                   }
            try:
                df = pd.concat([df, pd.DataFrame(data)])
            except NameError:
                df = pd.DataFrame(data)

    return df 

In [10]:
rens_kommune_data(kalund)   

,id,kommune,letter,party,name,answers
0,33,kalundborg,A,Socialdemokratiet,Gunver Jensen,111255531144343
0,33,kalundborg,A,Socialdemokratiet,Lars Kuhre Mortensen,141455543235335
0,33,kalundborg,A,Socialdemokratiet,Kirsten Rask,112544532225424
0,33,kalundborg,A,Socialdemokratiet,Hanne Olesen,221145442244434
0,33,kalundborg,A,Socialdemokratiet,Marianne Madsen,000000000000000
0,33,kalundborg,A,Socialdemokratiet,Aase Due,121155551153253
0,33,kalundborg,A,Socialdemokratiet,Carl Andersen,000000000000000
0,33,kalundborg,A,Socialdemokratiet,Sylvester Agbedoglo,000000000000000
0,33,kalundborg,A,Socialdemokratiet,Maja Hansen,111155533355134
0,33,kalundborg,A,Socialdemokratiet,Arne Jensen,221555454123343


### 1.6 Kør koden hen over alle kommunerne 
Tidligere scrapede i en liste over alle kommune*slugs*. Nu skal i skrive et loop som henter data, og derefter renser det for hver enkelt kommune i listen. Sørg for at gemme data fra de kommuner i har scrapet undervejs (evt med `pd.concat()`)

> **OBS:** i skal indlægge lidt forsinkelse af pythons kald til serveren i jeres script, ellers risikerer i at DOS'e DR (ikke godt). Brug derfor `sleep` hver gang i kalder serveren:
> ```python
> from time import sleep
> 
> sleep(2)
> kald_til_server()
> ```
> Sørg for at sleep bliver kaldet mindst en gang for hver request i laver.

In [17]:
from time import sleep 

for kom in kommuneslugs:
    sleep(2)
    print(kom)
    data = hent_kommune_data(kom)
    clean = rens_kommune_data(data)
    
    try:
        df = pd.concat([df, pd.DataFrame(data)])
    except NameError:
        df = pd.DataFrame(data)

        
df

albertslund
alleroed
assens
ballerup
billund
bornholm
broendby
broenderslev
dragoer
egedal
esbjerg
fanoe
favrskov
faxe
fredensborg
fredericia
frederiksberg
frederikshavn
frederikssund
furesoe
faaborg-midtfyn
gentofte
gladsaxe
glostrup
greve
gribskov
guldborgsund
haderslev
halsnaes
hedensted
helsingoer
herlev
herning
hilleroed
hjoerring
holbaek
holstebro
horsens
hvidovre
hoeje-taastrup
hoersholm
ikast-brande
ikke-oplyst


UnboundLocalError: local variable 'df' referenced before assignment